In [1]:
from pathlib import Path

In [2]:
import sys
import warnings

import datetime as dt

import numpy as np
import matplotlib.pyplot as plt

# Load the GMAT Python API
# sys.path.append(f'{Path.home()}/Downloads/GMAT-src-R2020a/GMAT-R2020a/GMAT-R2020a-Linux-x64/api/')
sys.path.append(f'C:/Work/Programme/GMAT_R2022a/GMAT/api/')
from load_gmat import *

from skyfield.api import EarthSatellite, load
from sgp4.api import Satrec, WGS72
from sgp4.model import wgs72, wgs84
from sgp4.conveniences import jday_datetime, UTC, sat_epoch_datetime, dump_satrec

import pandas as pd

import coord_skyfield
from coord_skyfield import ITRF2TEME, TEME2ITRF

from src.tlefit_equinoctial_eph_fd import *


## OPM Input
If you have a pre or post-launch OPM from your launch provider, enter the epoch and state vector in the cell below. This script expects a UTC timestamp and a state vector in the rotating ITRF (ECEF) coordinate frame. This is usually what is provided, but if you have something else, you can usually do some extra work to get to ECEF.

If you don't have an OPM or you're just experimenting, use the dummy OPM data we generated above.

Since we're using python lists, you can enter multiple epochs and state vectors to represent multiple satellites.

In [3]:
epochs = ["31 May 2024 13:20:20.000"]

#states = [([-3173.91430404, -6203.94723041, 802.14005485], [-0.99114021, 1.42336564, 7.44539408])]
#states = [([4425.943634402046, 1513.1108258432932, -5022.335898370971],[-4.576082632044403, -3.4947594109091738, -5.095486148173292])]

states = [([2128.164444925197, 104.52700298680319, -6525.30577021665],[-6.3570346092839845, -3.734255833115094, -2.1393941403847384])]

Loop through OPM satellites and use GMAT to propagate the spacecraft forward by 3 days using a high precision orbit propagator (Special Perturbations) to obtain an ephemeris we can later use to  fit a TLE

In [4]:
for idx, (t, state) in enumerate(zip(epochs, states)):
    print(idx)

    r, v = state[0], state[1]

    gmat.LoadScript("gmat/prelaunch_opm.script")

    sat = gmat.GetObject("Sat")
    sat.SetField("Epoch", t)
    sat.SetField("X", r[0])
    sat.SetField("Y", r[1])
    sat.SetField("Z", r[2])
    sat.SetField("VX", v[0])
    sat.SetField("VY", v[1])
    sat.SetField("VZ", v[2])

    # You will want to set appropriate values for drag and mass specific to your staellites
    sat.SetField("DragArea", 0.125) # m^2
    sat.SetField("DryMass", 22.5) # kg
    
    eph = gmat.GetObject("EphemerisFile1")
    eph.SetField("Filename", f'/EphemerisFile_Sat{idx}.e')

    gmat.RunScript()

0


Read the GMAT ephemeris files and save them as feather files.

In [5]:
for idx, t in enumerate(epochs):

    df = pd.read_fwf(f'C:/Work/Programme/GMAT_R2022a/GMAT/output/EphemerisFile_Sat{idx}.e', widths=(21, 24, 24, 24, 24, 24, 24), names=('time', 'x', 'y', 'z', 'xdot', 'ydot', 'zdot'), skiprows=15, skipfooter=4)
    df['timestamp'] = pd.Timestamp(epochs[idx], tz='UTC') + pd.to_timedelta(df.time, unit='s')
    df.to_feather(f'sat{idx}.fth')


Loop over the satellites' ephemeris we generated from GMAT and use it to fit a TLE.

In [6]:
for idx in range(len(states)):

    df = pd.read_feather(f'sat{idx}.fth')

    # Convert state vectors from ECEF to TEME
    t = df.timestamp
    ephemeris = [((row['x'], row['y'], row['z']), (row['xdot'], row['ydot'], row['zdot'])) for idx, row in df.iterrows()]

    ephemeris_teme = ITRF2TEME(t, ephemeris)
    ephemeris = ephemeris_teme
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        t = np.array([_t.to_pydatetime() for _t in t])

    # Run the fitter
    last_obs = 4320
    obs_stride = 1
    epoch_obs = 0
    lamda = 1e-3 + 1 # Interesting.The smaller number works, but diverges. This is better
    rms_epsilon = 0.002
    iterations, solve_sat, elements_coe, sigma, sigmas, dxs, bs, lamdas, b_epoch, b_new_epoch, b, P, A = \
    test_tle_fit_normalized_equinoctial(t, ephemeris, central_diff=True, last_obs=last_obs, obs_stride=obs_stride, epoch_obs=epoch_obs, lamda=False, rms_epsilon=rms_epsilon, debug=True)

    # Optionally thin the observations
    tt = t[::obs_stride]
    tephemeris = ephemeris[::obs_stride]

    if last_obs:
        tt = tt[:last_obs]
        tephemeris = tephemeris[:last_obs]

    print(f'Epoch: {tt[epoch_obs]}\n')
    print('\n'.join(exporter.export_tle(solve_sat.model)))
    print('\n')

Initial semi-major axis (a) = 6842.504 km
COE elements (original) = [6842.503782710278, 0.0034490738841777367, 1.704301397836347, 5.17869936708566, 1.7430924525736007, 2.9616747057271646, 1e-06]
Residuals at epoch time [ 1.46925710e+00  6.73826602e-01 -7.21732027e+00 -6.85287856e-03
  1.35451938e-02  2.03071928e-04]
Residual magnitudes at epoch time 7.39611, 0.0151814


#################### ITERATION 1 ####################

Condition number (A): 45906.459
Condition number (ATWA_acc): 588971.7892091068
Covariance a: 0.005 m
dx  [ 1.32652377e+01 -2.04596710e-03 -1.05144525e-03  4.33928874e-03
  1.20893014e-05 -1.00628104e-04  6.62155266e-03]
COE elements = [6855.7690203716975, 0.0012378400406727619, 1.7042528183296013, 5.178625497998073, 2.051221361533707, 2.65795895459392, 0.0066225526609072485]
EQN elements = (6855.7690203716975, 0.0007233880108566206, 0.0010044688905295408, 3.604620506946114, -1.0212057117347049, 0.513911907003667, 0.0066225526609072485)
Residual (b) = [ 2.45658952e+0

In [7]:
import math 
a, ecc, incl, raan, argp, mean_anomaly, bstar = elements_coe
R2D = 180/math.pi

print(a)
print(ecc)
print(incl*R2D)
print(raan*R2D)
print(argp*R2D)
print(mean_anomaly*R2D)

6854.958806137232
0.0008219023883144482
97.64357826052644
296.7147166345921
143.110951895176
126.45586274660816


In [8]:
display(dxs)
display(iterations)
display(sigmas)
display(lamdas)
display(b)

# iterations, solve_sat, elements_coe, sigma, sigmas, dxs, bs, lamdas, b_epoch, b_new_epoch, b, P, A

[array([ 1.32652377e+01, -2.04596710e-03, -1.05144525e-03,  4.33928874e-03,
         1.20893014e-05, -1.00628104e-04,  6.62155266e-03]),
 array([-7.97113817e-01, -5.77213474e-04, -2.01091437e-04, -4.26220577e-03,
         6.42464271e-05, -3.11756305e-06, -6.13251938e-03]),
 array([-1.30463762e-02, -9.89066997e-07,  5.59936001e-06, -6.61284762e-05,
         1.28017944e-07,  4.73704016e-07, -9.35933234e-05]),
 array([-5.40417041e-05, -1.50712731e-09,  1.03041911e-09, -2.65871927e-07,
        -2.06653382e-10, -2.01663855e-09, -5.51846462e-07])]

4

[115308.84182135249, 4574.285217876646, 68.16809434167043, 28.011527076124676]

[]

array([[-2.07357871e+02,  4.15812626e+02, -1.24001475e+01],
       [ 6.76976354e-02,  1.83295395e-02, -5.10132205e-01]])